In [2]:
import os
from pathlib import Path
from dotenv import load_dotenv

In [3]:
load_dotenv()
groq_api_key = os.environ.get("GROQ_API_KEY")

In [4]:
# Loading/Reading all the pdf documents

from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader

loader = DirectoryLoader(
    "../data/pdf_files",
    glob="**/*.pdf",
    loader_cls=PyPDFLoader
)

docs = loader.load()
len(docs)


56

In [5]:
# Splitting the Texts into Chunks

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    separators=["\n\n", "\n", " ", ""]
)

split_texts = text_splitter.split_documents(docs)
print(f"Created {len(split_texts)} chunks")

Created 98 chunks


In [7]:
# Embedding the split texts into vectors

from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# Creating the Vector Store (ChromaDB)

from langchain_community.vectorstores import Chroma

vectorstores = Chroma.from_documents(
    documents=split_texts,
    embedding=embeddings,
    persist_directory="../data/chroma_db"
)

print(f"✅ Vector store created with {len(split_texts)} embeddings")

✅ Vector store created with 98 embeddings


In [8]:
retriever = vectorstores.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}  # Return top 3 most relevant chunks
)

In [9]:
# Adding Memory

from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    output_key="answer"
)

C:\Users\Admin\AppData\Local\Temp\ipykernel_9756\2105759968.py:5: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [11]:
# Prompting

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert AI assistant specialized in construction tender analysis.

Your role is to help analyze tender documents and company capabilities to determine bid feasibility.

IMPORTANT RULES:
- Answer naturally and conversationally (never say "according to the document")
- Use the conversation history to understand context and follow-up questions
- Be specific with numbers, dates, and requirements
- If you don't have enough information, say so clearly
- Focus on actionable insights for decision-making
- Compare tender requirements against company capabilities when relevant

Remember: You're helping a construction company decide whether to bid on projects."""),
    
    MessagesPlaceholder(variable_name="chat_history"),
    
    ("human", """Context information from documents:
{context}

Question: {question}""")
])

from langchain_groq import ChatGroq

groq_llm = ChatGroq(
    groq_api_key=groq_api_key,
    model_name="llama-3.3-70b-versatile",
    temperature=0.1,  # Low temperature for factual responses
    max_tokens=1024
)

print("✅ Connected to Groq LLM (llama-3.3-70b-versatile)")

✅ Connected to Groq LLM (llama-3.3-70b-versatile)


In [12]:
# Build RAG Chain

from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

def format_docs(docs):
    """Format retrieved documents into a single string"""
    return "\n\n".join([doc.page_content for doc in docs])

def get_chat_history(memory):
    """Extract chat history from memory"""
    return memory.load_memory_variables({}).get("chat_history", [])

rag_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough(),
        "chat_history": lambda x: get_chat_history(memory)
    }
    | prompt
    | groq_llm
    | StrOutputParser()
)

In [13]:
def chat_with_memory(question):
    try:
        # Get answer from RAG chain
        answer = rag_chain.invoke(question)
        
        # Save to memory
        memory.save_context(
            {"input": question},
            {"answer": answer}
        )
        
        return answer
    
    except Exception as e:
        return f"Error generating response: {e}"

In [14]:
response1 = chat_with_memory("What is the Lekki Bridge project about?")
print(response1)

The Lekki Bridge project is a 2.4-kilometer reinforced concrete bridge construction project along the Lekki-Epe Expressway coastal corridor. The goal of the project is to reduce traffic congestion and improve coastal connectivity as part of the Lagos State Infrastructure Development Initiative. The project involves a range of activities, including detailed engineering design, site preparation, construction of the bridge foundation, erection of the superstructure, installation of drainage and safety systems, and more. The project is expected to commence on May 1, 2025, and be completed by April 30, 2027, with a 24-month defects liability period. 

To determine bid feasibility, we would need to assess your company's capabilities in terms of experience with similar projects, available resources, and expertise in areas like bridge construction, soil stabilization, and environmental impact mitigation. We should also consider the project's requirements, such as the 24-month contract period a

In [15]:
response2 = chat_with_memory("What are the key eligibility requirements for bidders?")
print(response2)

To be eligible to bid on the Lekki Bridge project, companies must meet certain key requirements. 

Firstly, from a registration perspective, bidders need to possess a valid Corporate Affairs Commission (CAC) registration certificate, a Tax Clearance Certificate for the current year (2025), evidence of registration with the Federal Inland Revenue Service (FIRS), a VAT registration certificate, and a Pension Commission (PenCom) compliance certificate.

From a technical standpoint, bidders must have a minimum of 10 years of operational experience in civil engineering construction and have successfully completed at least three bridge construction projects within the last 7 years.

These eligibility criteria are non-negotiable, and bidders must meet all of them to be considered for the project. 

Does your company meet these registration and technical requirements?


In [16]:
response3 = chat_with_memory("Does Elalan Construction meet these requirements?")
print(response3)

Based on the provided document, it appears that Elalan Construction Limited meets the key eligibility requirements for bidders on the Lekki Bridge project.

From a registration perspective, although the document doesn't explicitly mention the required registration certificates, it does establish Elalan Construction Limited as a legitimate company with a long history of operations in Nigeria, which suggests they likely have the necessary registrations.

From a technical standpoint, Elalan Construction Limited has over 23 years of experience in civil engineering construction, which exceeds the 10-year requirement. They also specialize in bridge construction, among other areas, and have a distinguished portfolio of over 180 successfully completed projects valued at over ₦120 billion cumulatively. This suggests they have the necessary technical expertise and experience to handle a project like the Lekki Bridge.

Additionally, the company's experience in handling complex, high-value infrast